In [0]:
catalogo = "medalhao"
silver_db_name = "silver"

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

consumidores_bronze_df = spark.table("medalhao.bronze.ft_consumidores")
pedidos_bronze_df = spark.table("medalhao.bronze.ft_pedidos")
pedidos_itens_bronze_df = spark.table("medalhao.bronze.ft_itens_pedidos")
pagamentos_bronze_df = spark.table("medalhao.bronze.ft_pagamentos_pedidos")
produtos_bronze_df = spark.table("medalhao.bronze.ft_produtos")
categoria_produtos_traducao_bronze_df = spark.table("medalhao.bronze.dm_categoria_produtos_traducao")
vendedores_bronze_df = spark.table("medalhao.bronze.ft_vendedores")
avaliacoes_pedidos_bronze_df = spark.table("medalhao.bronze.ft_avaliacoes_pedidos")
cotacao_dolar_bronze_df = spark.table("medalhao.bronze.dm_cotacao_dolar")


In [0]:
consumidores_bronze_df.printSchema()

In [0]:
consumidores_silver_df = (consumidores_bronze_df
                          .select(
                              F.col('customer_id').alias('id_consumidor'),
                              F.upper(F.col('customer_city')).alias('cidade'),
                              F.col('customer_zip_code_prefix').alias('prefixo_cep'),
                              F.upper(F.col('customer_state')).alias('estado'),
                              F.col('ingestion_timestamp').alias('data_ingestao')
                          )
                          .dropDuplicates(['id_consumidor'])
                          .withColumn('data_ingestao', F.current_timestamp()))


In [0]:
pedidos_bronze_df.printSchema()

pedidos_bronze_df = (pedidos_bronze_df.
                     withColumn('order_status', F.when(F.col('order_status') == 'canceled', 'cancelado').
                                            when(F.col('order_status') == 'shipped', 'enviado').
                                            when(F.col('order_status') == 'processing', 'em processamento').
                                            when(F.col('order_status') == 'invoiced', 'faturado').
                                            when(F.col('order_status') == 'delivered', 'entregue').
                                            when(F.col('order_status') == 'unavailable', 'indisponível').
                                            when(F.col('order_status') == 'created', 'criado').
                                            when(F.col('order_status') == 'approved', 'aprovado').
                                            otherwise("NÃO INFORMADO")))



In [0]:


pedidos_silver_df = (pedidos_bronze_df.
                     select(
                         F.col("order_id").alias("id_pedido"),
                         F.col("customer_id").alias("id_consumidor"),
                         F.col("order_status").alias("status"),
                         F.col("order_purchase_timestamp").alias("pedido_compra_timestamp"),
                         F.col("order_approved_at").alias("pedido_aprovado_timestamp"),
                         F.col("order_delivered_carrier_date").alias("pedido_carregado_timestamp"),
                         F.col("order_delivered_customer_date").alias("pedido_entregue_timestamp"),
                         F.col("order_estimated_delivery_date").alias("pedido_estimativa_entrega_timestamp"),
                         F.col("ingestion_timestamp").alias("data_ingestao"))
                     .withColumn("tempo_entrega_dias", F.date_diff(F.col("pedido_entregue_timestamp"), F.col("pedido_compra_timestamp")))

                     .withColumn("tempo_entrega_estimado_dias", F.date_diff(F.col
                     ("pedido_estimativa_entrega_timestamp"), F.col("pedido_compra_timestamp")))

                     .withColumn("diferenca_entrega_dias", F.col("tempo_entrega_dias")- F.col("tempo_entrega_estimado_dias"))

                     .withColumn("entrega_no_prazo", 
                                 F.when(F.col("diferenca_entrega_dias") <= 0, "Sim").
                                 when(F.col("status") != "entregue", "Não entregue").
                                 otherwise("Não"))

                    .withColumn('data_ingestao', F.current_timestamp()))



In [0]:

pedidos_itens_bronze_df.printSchema()
pedidos_itens_silver_df = (
    pedidos_itens_bronze_df
    .withColumnRenamed('order_id', 'id_pedido')
    .withColumnRenamed('order_item_id', 'id_item')
    .withColumnRenamed('product_id', 'id_produto')
    .withColumnRenamed('seller_id', 'id_vendedor')
    .withColumnRenamed('product_category_name', 'categoria_produto')
    .withColumnRenamed('price', 'preco_BRL')
    .withColumnRenamed('freight_value', 'preco_frete')
    .withColumnRenamed('ingest_data', 'data_ingestao')
    .withColumn('preco_BRL', F.col('preco_BRL').cast(T.DecimalType(12,2)))
    .withColumn('preco_frete', F.col('preco_frete').cast(T.DecimalType(12,2)))
    .withColumn('data_ingestao', F.current_timestamp())
)


In [0]:
pagamentos_bronze_df.printSchema()

pagamentos_bronze_df = (
    pagamentos_bronze_df
    .withColumn(
        "payment_type",
        F.when(F.col("payment_type") == "credit_card", "Cartão de Crédito")
         .when(F.col("payment_type") == "boleto", "Boleto")
         .when(F.col("payment_type") == "voucher", "Voucher")
         .when(F.col("payment_type") == "debit_card", "Cartão de Débito")
         .otherwise("Outro")
    )
)

In [0]:
pagamentos_silver_df = (
    pagamentos_bronze_df
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("payment_sequential").alias("codigo_pagamento"),
        F.col("payment_type").alias("forma_pagamento"),
        F.col("payment_installments").alias("parcelas"),
        F.col("payment_value").alias("valor_pagamento"),
        F.col("ingestion_timestamp").alias("data_ingestao"),
    )
    .withColumn('valor_pagamento', F.col('valor_pagamento').cast(T.DecimalType(12,2)))
    .withColumn('data_ingestao', F.current_timestamp())
)


In [0]:
produtos_bronze_df.printSchema()

df_produtos_silver = (produtos_bronze_df.
                      select(
                            F.col("product_id").alias("id_produto"),
                            F.col("product_category_name").alias("categoria_produto"),
                            F.col("product_length_cm").alias("comprimento_centimetros"),
                            F.col("product_height_cm").alias("altura_centimetros"),
                            F.col("product_width_cm").alias("largura_centimetros"),
                            F.col("product_weight_g").alias("peso_produto_gramas"),
                            F.col("ingestion_timestamp").alias("data_ingestao"),
                      )
                      .withColumn('data_ingestao', F.current_timestamp())
)

In [0]:
categoria_produtos_traducao_silver_df = (
    categoria_produtos_traducao_bronze_df
    .select(
        F.col("product_category_name").alias("nome_produto_pt"),
        F.col("product_category_name_english").alias("nome_produto_en"),
        F.col("ingestion_timestamp").alias("data_ingestao"),
    )
    .withColumn('data_ingestao', F.current_timestamp())
)

In [0]:
vendedores_silver_df = (vendedores_bronze_df
                        .select(
                            F.col("seller_id").alias("id_vendedor"),
                            F.col("seller_zip_code_prefix").alias("prefixo_cep"),
                            F.upper(F.col("seller_city")).alias("cidade"),
                            F.upper(F.col("seller_state")).alias("estado"),
                            F.col("ingestion_timestamp").alias("data_ingestao"),
                        )
                        .withColumn('data_ingestao', F.current_timestamp()))
                        

### Regras de validação em ft_avaliacoes_pedidos
* ID não pode ser **nulo**
* Data tem que estar no formato: yyyy-MM-dd HH:mm:ss.SSSS
* Data não pode ser **nulo**
* ID tem que ser validado pelo regex: ^[0-9a-f]{32}$

In [0]:
nulos_df = avaliacoes_pedidos_bronze_df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in avaliacoes_pedidos_bronze_df.columns
]).display()

In [0]:
avaliacoes_pedidos_bronze_df.printSchema()

In [0]:
#Separando dados válidos e inválidos
avaliacoes_pedidos_bronze_validados_df = (
    avaliacoes_pedidos_bronze_df
    .withColumn("flag_review_creation_date_notnull", F.col("review_creation_date").isNotNull())
    .withColumn("flag_review_answer_timestamp_notnull", F.col("review_answer_timestamp").isNotNull())
    .withColumn("flag_order_id_notnull", F.col("order_id").isNotNull())
    .withColumn("flag_review_id_notnull", F.col("review_id").isNotNull())
    .withColumn("flag_review_creation_date_format", F.col("review_creation_date").rlike(r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"))
    .withColumn("flag_review_answer_timestamp_format", F.col("review_answer_timestamp").rlike(r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"))
    .withColumn("flag_order_id_regex", F.col("order_id").rlike(r"^[0-9a-f]{32}$"))
    .withColumn("flag_review_id_regex", F.col("review_id").rlike(r"^[0-9a-f]{32}$"))
    .withColumn(
        "validacao",
        F.when(
            F.col("flag_review_creation_date_notnull") &
            F.col("flag_review_answer_timestamp_notnull") &
            F.col("flag_order_id_notnull") &
            F.col("flag_review_id_notnull") &
            F.col("flag_review_creation_date_format") &
            F.col("flag_review_answer_timestamp_format") &
            F.col("flag_order_id_regex") &
            F.col("flag_review_id_regex"),
            "OK"
        ).otherwise("NO")
    )
    .withColumn('data_ingestao', F.current_timestamp())
)

In [0]:
avaliacoes_pedidos_silver_validos_df = (
    avaliacoes_pedidos_bronze_validados_df
    .filter(F.col("validacao") == "OK")
    .select(
        F.col("review_id").alias("id_avaliacao"),
        F.col("order_id").alias("id_pedido"),
        F.col("review_score").alias("avaliacao"),
        F.col("review_comment_title").alias("titulo_comentario"),
        F.col("review_comment_message").alias("comentario"),
        F.col("review_creation_date").alias("data_comentario"),
        F.col("review_answer_timestamp").alias("data_resposta"),
        F.col("ingestion_timestamp").alias("data_ingestao"),
    )
    .withColumn('avaliacao', F.col('avaliacao').cast(T.IntegerType()))
    .withColumn('data_resposta', F.to_date(F.col('data_resposta'), 'yyyy-MM-dd HH:mm:ss'))
    .withColumn('data_comentario', F.to_date(F.col('data_comentario'), 'yyyy-MM-dd HH:mm:ss'))
    .withColumn('data_ingestao', F.current_timestamp())
)

avaliacoes_pedidos_silver_invalidos_df = avaliacoes_pedidos_bronze_validados_df.filter(F.col("validacao") != "OK")


In [0]:
#Aplicndo regras de qualidade mensuráveis
colunas_auditoria_avaliacao = ["porcentagem_validos", "porcentagem_invalidos"]
auditoria_avaliacao_df = spark.createDataFrame([(
    ((avaliacoes_pedidos_silver_validos_df.count() / avaliacoes_pedidos_bronze_df.count()) * 100,
     (avaliacoes_pedidos_silver_invalidos_df.count() / avaliacoes_pedidos_bronze_df.count() * 100)
     )
)], colunas_auditoria_avaliacao)

display(auditoria_avaliacao_df)

In [0]:
pedidos_orfaos = (pedidos_silver_df
                  .join(
                      consumidores_silver_df,
                      on="id_consumidor",
                      how="leftanti",
                  ))
print(pedidos_orfaos.count())

In [0]:
pedidos_item_orfaos = (pedidos_itens_silver_df
                       .join(
                           pedidos_silver_df,
                           on="id_pedido",
                           how="leftanti",
                       ))

print(pedidos_item_orfaos.count())

In [0]:
def remover_orfaos(df_silver, df_orfao, match):
    if (df_orfao.count() > 0):
        return df_silver
    
    return (df_silver.join(
        df_orfao,
        on=f"{match}",
        how="leftanti",
    ))

pedidos_itens_silver_df = remover_orfaos(pedidos_itens_silver_df, pedidos_item_orfaos, "id_pedido")
pedidos_silver_df = remover_orfaos(pedidos_silver_df, pedidos_orfaos, "id_consumidor")

In [0]:
from pyspark.sql.window import Window

w = Window.orderBy(F.asc("dataHoraCotacao"))
cotacao_dolar_silver_df = (
    cotacao_dolar_bronze_df
    .withColumn("dataHoraCotacao", F.to_date(F.col("dataHoraCotacao"), "yyyy-MM-dd"))
    .withColumn("next_day", F.lead("dataHoraCotacao", 1).over(w))
    .withColumn("diff_days", F.date_diff("next_day", "dataHoraCotacao"))
)

cotacao_dolar_silver_1_df = (cotacao_dolar_silver_df.filter(F.col("diff_days") > 1))

cotacao_dolar_silver_2_df = (cotacao_dolar_silver_1_df.
                           withColumn("array_date", F.sequence(F.lit(1), F.col("diff_days")- 1)))

cotacao_dolar_silver_3_df = (cotacao_dolar_silver_2_df.
                           withColumn("day_to_add", F.explode(F.col("array_date"))))

cotacao_dolar_silver_4_df = (cotacao_dolar_silver_3_df.
                           withColumn("dataHoraCotacao", F.date_add(F.col("dataHoraCotacao"), F.col("day_to_add"))))

cotacao_dolar_silver_df = (cotacao_dolar_silver_df.
                           select(["cotacaoCompra", "dataHoraCotacao"])
                           .union(cotacao_dolar_silver_4_df.
                           select(["cotacaoCompra", "dataHoraCotacao"]))
                           .orderBy("dataHoraCotacao"))      


In [0]:
total_items_value_df = (
    pedidos_itens_bronze_df
    .groupBy("order_id")
    .agg(F.sum(F.col("price")).alias("total_items_value"))
)

total_freight_value_df = (
    pedidos_itens_bronze_df
    .groupBy("order_id")
    .agg(F.sum(F.col("freight_value")).alias("total_freight_value"))
)

total_payment_value_df = (
    pagamentos_bronze_df
    .groupBy("order_id")
    .agg(F.sum(F.col("payment_value")).alias("total_payment_value"))
)

In [0]:


pedido_total_silver = (
    consumidores_bronze_df
    .join(
        pedidos_bronze_df,
        on="customer_id",
        how="inner",
    )
    .join(
        pagamentos_bronze_df,
        on="order_id",
        how="inner"
    )
    .join(
        cotacao_dolar_bronze_df,
        on=F.date_trunc("day", F.col("order_purchase_timestamp")) == F.date_trunc("day", F.col("dataHoraCotacao")),
        how="inner"
    )
    .groupBy(
        F.col("order_id"),
        F.col("customer_id"),
        F.col("order_status"),
        F.to_date(F.col("order_purchase_timestamp"), 'yyyy-MM-dd').alias("data_pedido"),
        F.col("cotacaoCompra")
    )
    .agg(
        F.sum(F.col("payment_value")).alias("valor_total_pago_brl")
    )
    .withColumn("valor_total_pago_usd", F.col("valor_total_pago_brl") / F.col("cotacaoCompra"))
    .withColumn("data_ingestao", F.current_timestamp())
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("customer_id").alias("id_consumidor"),
        F.col("order_status").alias("status"),
        F.col("data_pedido"),
        F.col("valor_total_pago_brl"),
        F.col("valor_total_pago_usd")
    )
)


In [0]:
def write_df_silver(df, nome_tabela, tipo="valido"):
   
    try:
        # Validação: arquivo vazio
        if df.count() == 0:
            raise ValueError(f"o data frame está vazio")

        # Escrita no formato Delta
        df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.{nome_tabela}")

        if tipo == "valido":
            print(f"✅ Tabela silver.{nome_tabela} criada com sucesso!\n")
        elif tipo == "invalido":
            print(f"⚠️ Tabela silver.{nome_tabela} criada para auditoria ({df.count()} registros inválidos)")
        elif tipo == "auditoria":
            print(f"⚠️ Tabela silver.{nome_tabela} criada para auditoria")

    except Exception as e:
        print(f"Erro ao processar {nome_tabela}: {str(e)}")

In [0]:
write_df_silver(consumidores_silver_df, "consumidores_silver")
write_df_silver(pedidos_silver_df, "pedidos_silver")
write_df_silver(pedidos_itens_silver_df, "pedidos_itens_silver")
write_df_silver(pagamentos_silver_df, "pagamentos_silver")
write_df_silver(df_produtos_silver, "produtos_silver")
write_df_silver(categoria_produtos_traducao_silver_df, "categoria_produtos_traducao_silver")
write_df_silver(vendedores_silver_df, "vendedores_silver")
write_df_silver(avaliacoes_pedidos_silver_validos_df, "avaliacoes_pedidos_validos_silver")
write_df_silver(avaliacoes_pedidos_silver_invalidos_df, "avaliacoes_pedidos_invalidos_silver", "invalido")
write_df_silver(auditoria_avaliacao_df, "auditoria_avaliacao_silver", "auditoria")
write_df_silver(cotacao_dolar_silver_df, "cotacao_dolar_silver")